# RAG Evaluation

This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.


### What is RAG?

When using a Large Language Model (LLM) to answer questions about a particular field of knowledge, two issues may appear:
- the LLM not having knowledge of domain-specific information due to it not being available in its training data
- the risk of hallucination: when asked to give an answer on an unknown field, the LLM may hallucinate a credible answer.

These two problems can be tempered by fine-tuning the LLM on the said field of knowledge
However, fine-tuning can be costly, and, when done repeatedly (e.g. to address changes in data), leads to "model shift" _(this is when the model's behavior changes in ways that are not desirable)_.

RAG (Retrieval Augmented Generation) does not require model fine-tuning. Instead, RAG works by enhancing an LLM with access to additional context retrieved from relevant data, so that it can generate a better-informed response.

RAG systems consists of two subsystems:

1. A Retriever system, functioning like an internal search engine, identifies the most relevant documents from the knowledge base to respond to the query.
2. A Reader Language Model (LLM) reads these retrieved documents to produce an answer to the user’s query.

Here is the workflow, with paths for system tuning noted in blue:

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_graph.png" height="700">


> 💡 As you can see, there are many steps to tune in this architecture: tuning the system properly will yield significant performance gains.

RAG systems have plentiful options for improvement: you can tune your system along the way to improve results beyond a baseline model.

But this is useless if you cannot monitor the impact of your changes on the system's performance!

__Let's dig into it and start building our evaluation pipeline!__ First, we install the required model dependancies.

In [1]:
!pip install -q torch transformers transformers langchain sentence-transformers faiss-gpu openpyxl openai


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
%reload_ext autoreload
%autoreload 2
%reload_ext dotenv
%dotenv

In [3]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from langchain_core.language_models import BaseChatModel
from datasets import Dataset

pd.set_option("display.max_colwidth", None)

### Load your knowledge base

In [4]:
import datasets

ds = datasets.load_dataset("A-Roucher/huggingface_doc", split="train")

## 1 Build a synthetic dataset for evaluation
We first build a synthetic dataset of questions and associated contexts.

The idea is to randomly get elements from our knowledge base, and ask a LLM to generate questions based on these documents.

In [5]:
LOAD_PREGENERATED_EVALUATION_DATASET = True

if LOAD_PREGENERATED_EVALUATION_DATASET:
    eval_dataset = datasets.load_dataset("A-Roucher/huggingface_doc_qa_eval")

### Prepare source documents

In [6]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.docstore.document import Document as LangchainDocument

    langchain_docs = [
        LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
        for doc in tqdm(ds)
    ]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=200,
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in langchain_docs:
        docs_processed += text_splitter.split_documents([doc])

### Setup agents for question generation

In [7]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    from langchain.prompts import ChatPromptTemplate
    from langchain.chat_models import ChatOpenAI

    QA_generation_prompt = """
    Your task is to write a factoid question and an answer given a context.
    Your factoid question should be answerable with a specific, concise piece of factual information from the context.
    Your factoid question should be formulated in the same style as questions users could ask in a search engine.
    This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

    Provide your answer as follows:

    Output:::
    Factoid question: (your factoid question)
    Answer: (your answer to the factoid question)

    Now here is the context.

    Context: {context}\n
    Output:::"""

    chat_model = ChatOpenAI(model="gpt-4-1106-preview")
    QA_generation_prompt = ChatPromptTemplate.from_template(QA_generation_prompt)
    QA_generation_agent = QA_generation_prompt | chat_model

In [8]:
import random

if not LOAD_PREGENERATED_EVALUATION_DATASET:
    N_GENERATIONS = 30

    print(f"Generating {N_GENERATIONS} QA couples...")
    outputs = []
    for context in tqdm(random.sample(langchain_docs, N_GENERATIONS)):
        # Generate QA couple
        output_QA_couple = QA_generation_agent.invoke({"context": context.page_content}).content
        try:
            question = output_QA_couple.split("Factoid question: ")[1].split("Answer: ")[0]
            answer = output_QA_couple.split("Answer: ")[1]
            outputs.append(
                {
                    "context": context.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": context.metadata["source"],
                }
            )
        except:
            continue

### Setup critique agents
We now build and run the critique agents that will filter out questions generated by the previous LLM agent.

In [9]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    question_relatedness_critique_prompt = """
    You will be given a context and a question.
    Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
    Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

    Provide your answer as follows:

    Answer:::
    Evaluation: (your rationale for the rating)
    Total rating: (your rating)

    Now here are the question and context.

    Question: {question}\n
    Context: {context}\n
    Answer::: """

    question_relevance_critique_prompt = """
    You will be given a question.
    Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
    Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

    Rating must be minimum, i.e. 1, if the questions refers to a particular setting, like 'in the context' or 'in the document'.

    Provide your answer as follows:

    Answer:::
    Evaluation: (your rationale for the rating)
    Total rating: (your rating)

    Now here is the question.

    Question: {question}\n
    Answer::: """

    question_standalone_critique_prompt = """
    You will be given a question.
    Your task is to provide a 'total rating' representing how context-independant this question is.
    Give your answer on a scale of 1 to 5, where 1 means that the question only makes sense in a specific context, and 5 means that the question makes sense by itself.
    For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
    The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

    Provide your answer as follows:

    Answer:::
    Evaluation: (your rationale for the rating)
    Total rating: (your rating)

    Now here is the question.

    Question: {question}\n
    Answer::: """

    question_relatedness_critique_prompt = ChatPromptTemplate.from_template(
        question_relatedness_critique_prompt
    )
    question_relatedness_critique_agent = question_relatedness_critique_prompt | chat_model

    question_relevance_critique_prompt = ChatPromptTemplate.from_template(
        question_relevance_critique_prompt
    )
    question_relevance_critique_agent = question_relevance_critique_prompt | chat_model

    question_standalone_critique_prompt = ChatPromptTemplate.from_template(
        question_standalone_critique_prompt
    )
    question_standalone_critique_agent = question_standalone_critique_prompt | chat_model

In [10]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    print("Generating critique for each QA couple...")
    for output in tqdm(outputs):
        # Critique the generated QA couple
        question_relatedness_evaluation = question_relatedness_critique_agent.invoke(
            {"context": output["context"], "question": output["question"]}
        ).content
        question_relevance_evaluation = question_relevance_critique_agent.invoke(
            {"question": output["question"]}
        ).content
        question_standalone_evaluation = question_standalone_critique_agent.invoke(
            {"question": output["question"]}
        ).content

        try:
            relatedness_score = int(question_relatedness_evaluation.split("Total rating: ")[1][0])
            relatedness_eval = question_relatedness_evaluation.split("Total rating: ")[0].split(
                "Evaluation: "
            )[1]
            relevance_score = int(question_relevance_evaluation.split("Total rating: ")[1][0])
            relevance_eval = question_relevance_evaluation.split("Total rating: ")[0].split(
                "Evaluation: "
            )[1]
            standalone_score = int(question_standalone_evaluation.split("Total rating: ")[1][0])
            standalone_eval = question_standalone_evaluation.split("Total rating: ")[0].split(
                "Evaluation: "
            )[1]
            output.update(
                {
                    "relatedness_score": relatedness_score,
                    "relatedness_eval": relatedness_eval,
                    "relevance_score": relevance_score,
                    "relevance_eval": relevance_eval,
                    "standalone_score": standalone_score,
                    "standalone_eval": standalone_eval,
                }
            )
        except:
            continue

Filter down questions on the chosen criteria:

In [11]:
if not LOAD_PREGENERATED_EVALUATION_DATASET:
    import pandas as pd

    pd.set_option("display.max_colwidth", None)

    generated_questions = pd.DataFrame.from_dict(outputs)
    display(generated_questions.head())
    generated_questions = generated_questions.loc[
        (generated_questions["relatedness_score"] >= 4)
        & (generated_questions["relevance_score"] >= 3)
        & (generated_questions["standalone_score"] >= 4)
    ]
    generated_questions.to_excel("qa_eval.xlsx")
    display(generated_questions.head())

    eval_dataset = datasets.Dataset.from_pandas(
        generated_questions, split="train", preserve_index=False
    )

Now the synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

# 2 Run tests

## 2.1 Build RAG System

### Building the vector database: preprocessing

- In this part, __we split the documents from our knowledge base into smaller chunks__ which will be the snippets on which the reader LLM will base its answer.
- The goal is to have semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting each idea.

> We use Langchain's `RecursiveCharacterTextSplitter`, which makes efficient use of code language detection to make better splits the lenght function we use is not the count of characters, but the count of tokens.

💡 To help you tune this step, [this space](https://huggingface.co/spaces/A-Roucher/chunk_visualizer) lets you visualize how different chunking options affect the chunks you get.

In [12]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

  0%|          | 0/2647 [00:00<?, ?it/s]

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

### Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns the most relevant documents from your knowledge base.

> For the knowledge base, we use Langchain vector databases since __it offers a convenient [FAISS](https://github.com/facebookresearch/faiss) index and allows us to keep document metadata throughout the processing__.

🛠️ __Options included:__

- Tune the chunking method:
    - Size of the chunks
    - Method: split on different separators, use [semantic chunking](https://python.langchain.com/docs/modules/data_connection/document_transformers/semantic-chunker)...
- Change the embedding model

In [14]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

### Reader - LLM 💬

In this part, the __LLM Reader reads the retrieved documents to formulate its answer.__

🛠️ Here we tried the following options to improve results:
- Switch reranking on/off
- Change the reader model

In [15]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context, 
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [16]:
import requests
import os

API_URL = "https://qb2fcspon657hfe4.us-east-1.aws.endpoints.huggingface.cloud"

headers = {
    "Authorization": f"Bearer {os.getenv('HUGGINGFACEHUB_API_TOKEN')}",
    "Content-Type": "application/json",
}


def get_llm_answer(question):
    payload = {
        "inputs": question,
        "max_new_tokens": 3000,
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [17]:
get_llm_answer("What is the capital of France?")

{'error': 'Bad Gateway'}

In [18]:
from ragatouille import RAGPretrainedModel


def answer_with_rag(
    question: str,
    llm: BaseChatModel,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"{str(i)}: " + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(context=context, question=question)

    # Redact an answer
    output = llm(final_prompt)
    try:
        answer = output[0]["generated_text"]
    except:
        # Retry with shorter contect
        context = "\nExtracted documents:\n"
        context += "".join([f"{str(i)}: " + doc for i, doc in enumerate(relevant_docs[:-1])])

        final_prompt = RAG_PROMPT_TEMPLATE.format(context=context, question=question)

        # Redact an answer
        output = llm(final_prompt)
        answer = output[0]["generated_text"]

    return answer, relevant_docs

# 2. Benchmarking the RAG system

Since there are so many options to tune with a big impact on parameters, benchmarking the RAG system is crucial.

But to do so we will need:
- an evaluation dataset
- an evaluator to compute the accuracy of our system.

➡️ It turns out, we can use LLMs to help us all along the way!
- the evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
- the evaluation will then be run on this synthetic dataset by a LLM-as-a-judge agent 🤖.

In [19]:
import json
from ragatouille import RAGPretrainedModel


def run_rag_tests(
    eval_dataset: Dataset,
    llm: BaseChatModel,
    knowledge_index: FAISS,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
):
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []
    for example in tqdm(eval_dataset["train"]):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(question, llm, knowledge_index, reranker=reranker)
        if verbose:
            print(":::::::")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        outputs.append(
            {
                "question": question,
                "true_answer": example["answer"],
                "source_doc": example["source_doc"],
                "generated_answer": answer,
                "retrieved_docs": [doc for doc in relevant_docs],
            }
        )

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [20]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [21]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
evaluator_name = "GPT4"


def evaluate_answers(
    answer_path: str,
    eval_chat_model: BaseChatModel,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    try:  # load previous generations if they exist
        with open(answer_path, "r") as f:
            answers = json.load(f)
    except:
        answers = []

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_result = eval_chat_model.invoke(eval_prompt)
        feedback, score = [item.strip() for item in eval_result.content.split("[RESULT]")]
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with open(answer_path, "w") as f:
            json.dump(answers, f)

/home/ubuntu/venv/ml2/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
RAG_SETTINGS = {}
for chunk_size in [300]:
    for embeddings in ["thenlper/gte-small"]:
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader:mixtral-8x7b"

            print(f"Running evaluation for {settings_name}:")
            output_file_name = f"./output/rag_{settings_name}.json"

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            if rerank:
                reranker = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
            else:
                reranker = None
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=get_llm_answer,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                eval_chat_model,
                evaluator_name,
                evaluation_prompt_template,
            )

Running evaluation for chunk:400_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b:
Loading knowledge base embeddings...
Running RAG...


  0%|          | 0/67 [00:00<?, ?it/s]

Running evaluation...


  0%|          | 0/67 [00:00<?, ?it/s]

Running evaluation for chunk:400_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b:
Loading knowledge base embeddings...
Running RAG...


...

  0%|          | 0/67 [00:00<?, ?it/s]

Running evaluation for chunk:300_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader:mistral-7b:
Loading knowledge base embeddings...
Running RAG...


  0%|          | 0/67 [00:00<?, ?it/s]

Running evaluation...


  0%|          | 0/67 [00:00<?, ?it/s]

## 2.3 Visualize results

In [23]:
import glob

results = []
for file in glob.glob("output/*.json"):
    print(file)
    with open(file, "r") as f:
        sub_df = pd.read_json(f)
        sub_df["settings"] = file.split("rag_")[1].split(".json")[0]
        results.append(sub_df)

results_df = pd.concat(results)

output/rag_chunk:512_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b.json
output/rag_chunk:300_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b.json
output/rag_chunk:400_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader:mistral-7b.json
output/rag_chunk:400_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader:mistral-7b.json
output/rag_chunk:400_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b.json
output/rag_chunk:300_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader:mistral-7b.json
output/rag_chunk:400_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b.json
output/rag_chunk:512_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b.json
output/rag_chunk:300_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b.json
output/rag_chunk:300_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader:mistral-7b.json


In [24]:
def interpret_result(x):
    try:
        return (int(x) - 1) / 4
    except:
        return 0


results_df["normalized_score"] = results_df["eval_score_GPT4"].apply(interpret_result)

In [25]:
scores = results_df.groupby("settings")["normalized_score"].mean().sort_values() * 100
display(scores)
scores = scores.rename(
    index={
        "chunk:1500_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b": "Baseline RAG",
        "chunk:500_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b": "+ Tune chunk size",
        "chunk:500_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b": "+ Rerank results",
        "chunk:500_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader:mistral-7b": "+ Better embeddings",
        "chunk:700_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader:mixtral-8x7b": "+ Better reader LLM",
    }
)
scores = scores.loc[
    [
        "Baseline RAG",
        "+ Tune chunk size",
        "+ Rerank results",
        "+ Better embeddings",
        "+ Better reader LLM",
    ]
]

settings
chunk:512_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b       77.985075
chunk:400_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader:mistral-7b    79.477612
chunk:300_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b       79.850746
chunk:400_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader:mistral-7b     82.462687
chunk:512_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b        83.208955
chunk:400_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b        83.582090
chunk:400_embeddings:thenlper~gte-small_rerank:False_reader:mistral-7b       84.701493
chunk:300_embeddings:BAAI~bge-base-en-v1.5_rerank:True_reader:mistral-7b     85.447761
chunk:300_embeddings:BAAI~bge-base-en-v1.5_rerank:False_reader:mistral-7b    85.820896
chunk:300_embeddings:thenlper~gte-small_rerank:True_reader:mistral-7b        86.567164
Name: normalized_score, dtype: float64

KeyError: "None of [Index(['Baseline RAG', '+ Tune chunk size', '+ Rerank results',\n       '+ Better embeddings', '+ Better reader LLM'],\n      dtype='object', name='settings')] are in the [index]"

In [ ]:
import plotly.express as px

fig = px.bar(
    scores,
    color=scores,
    text=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="Accuracy of different RAG configurations",
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.0f}", textposition="outside")
fig.show()

<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_options_accuracy.png" height="500">